In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241204_01_RDD_API")
spark = SparkContext(conf=conf).getOrCreate()
spark

24/12/06 18:03:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local appName=241204_01_RDD_API>

In [2]:
foods = spark.parallelize([ "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면" ] )
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [3]:
#모두 메모리에 올립니다.
foods.collect() #take() show()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [4]:
#값을 기준으로 카운트 : 음식별 개수 세기
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [5]:
#상위 3개만 가져와
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [6]:
#처음 1개 가져오기
foods.first()

'짜장면'

In [7]:
#RDD 갯수
foods.count()

12

In [8]:
#중복 제거한 목록
#Transformation 연산
fd_dist = foods.distinct()
fd_dist

PythonRDD[9] at RDD at PythonRDD.scala:53

In [9]:
#action 연산 : Result
fd_dist.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [10]:
#워커노드에서 실행하는 기능
foods.foreach(lambda x : print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


In [11]:
foods.foreach(lambda x : x)

# Narrow Operation  1:1 연산 
filter(), map(), flatMap(), sample(), union()

In [12]:
sample_rdd = spark.parallelize([1,2,3,4,5])

In [13]:
sample_rdd2 = sample_rdd.map(lambda x : x+2) #d+2, 2+2, 3+2....

In [14]:
sample_rdd2.collect()

[3, 4, 5, 6, 7]

In [15]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [16]:
moviesRDD = spark.parallelize(movies)

In [17]:
mapMovies = moviesRDD.map(lambda x: x.split(" "))

In [18]:
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [19]:
flatMapMovies = moviesRDD.flatMap(lambda x:x.split(" ") )
flatMapMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [20]:
filteredMovies = flatMapMovies.filter(lambda x: x!="매트릭스")
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [21]:
#집합연산
num1 = spark.parallelize([1,2,3,4,5])
num2 = spark.parallelize([4,5,6,7,8,9,10])

In [22]:
#intersection
num1.intersection(num2).collect()

[4, 5]

In [23]:
#union, subtract

# sample(withReplacement, fraction, seed)

일부를 샘플링해서 추출한다.
```
withReplacement : 비복원, 복원 추출
fraction : 기대값
seed : 난수추출을 위한 시드값 - 재현 가능
```

In [24]:
numlist = num1.union(num2)
numlist.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [25]:
numlist.sample(True, 0.5).collect()

[4, 5, 7, 9, 10]

In [26]:
numlist.sample(True, 0.3).collect()

[3, 4, 4, 5, 10]

In [27]:
numlist.sample(False, 0.7).collect() #재현이 안되는 샘플

[2, 3, 5, 6, 7, 8, 9]

In [28]:
numlist.sample(True, 0.5, seed=42).collect() #재현 가능한 샘플

[5, 6, 6]

# wide transformation
groupby reduce.. 

In [29]:
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [30]:
foodsGroup = foods.groupBy(lambda x: x[0])
res = foodsGroup.collect()

In [31]:
for (k,v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [32]:
spark.stop()